In [ ]:
!pip install zhon

In [ ]:
import math
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as Data
import random
import time
import string
import zhon

In [ ]:
from zhon.hanzi import punctuation

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# torch.cuda.set_device(0)

In [ ]:
with open('train.txt', 'r') as file:
  train = file.readlines()
with open('test.txt', 'r') as file:
  test = file.readlines()
with open('dev.txt', 'r') as file:
  dev = file.readlines()

In [ ]:


# 主要处理英文部分，1.有的单词标点和单词相连接，进行分离处理，2.字母小写
# 分离句子d中的所有单词和标点
def splitw(d):
  
  res = []
  for w in d:
    if w in string.punctuation or w == '...' or w == '..':
      res.append(w[0])
      continue
    p = []
    while w and w[0] in string.punctuation:
      res.append(w[0])
      w = w[1:]
    while w and w[-1] in string.punctuation:
      p.append(w[-1])
      w = w[:-1]
    if w:
      res.append(w)
    res.extend(p)

  return res
def datapre(data, maxen=50, rp = True):
  data1 = [d[:-1] for d in data]
  data2 = [d.split() for d in data1]
  datazh = [d for i, d in enumerate(data2) if i % 2 == 0]
  dataen = [d for i, d in enumerate(data2) if i % 2 == 1]
  dataen1 = [splitw(d) for d in dataen] 
  dataen2 = [[w.lower() for w in d] for d in dataen1]

  # 默认去掉所有标点，否则标点符号统一化
  if rp:
    dataen3 = [[w for w in d if w not in string.punctuation] for d in dataen2]
    datazh1 = [[w for w in d if w not in punctuation] for d in datazh]
  else:
    dataen3 = [["." if w in string.punctuation else w for w in d] for d in dataen2]
    datazh1 = [["." if w in punctuation else w for w in d] for d in datazh]

  # 利用padding填充数据，有一些超长的句子就省略了
  # maxen = 50
  idx = [i for i, d in enumerate(dataen3) if len(d)<=50]
  dataen4 = [dataen3[i] for i in idx]
  datazh2 = [datazh1[i] for i in idx]
  maxzh = max(len(d) for d in datazh2)
  enpad = [d + ['<padding>'] * (maxen - len(d)) for d in dataen4]
  zhpad = [d + ['<padding>'] * (maxzh - len(d)) for d in datazh2]

  # 为句子补上起始符和终止符
  enfinal = [['<SOS>'] + d + ['<EOS>'] for d in enpad]
  zhfinal = [['<SOS>'] + d + ['<EOS>'] for d in zhpad]

  return enfinal, zhfinal


In [ ]:
# 将词汇数据处理为id数据
tr_en, tr_zh = datapre(train)
ts_en, ts_zh = datapre(test)
enlist, zhlist = [], []
for l in tr_en + ts_en:
  enlist += l
for l in tr_zh + ts_zh:
  zhlist += l 
en_vocab_size = len(set(enlist))
zh_vocab_size = len(set(zhlist))
en_id2char = list(set(enlist))
zh_id2char = list(set(zhlist))
en_char2id = {c:i for i,c in enumerate(en_id2char)}
zh_char2id = {c:i for i,c in enumerate(zh_id2char)}

# 将<padding>的索引指向0
eni = en_char2id['<padding>']
zhi = zh_char2id['<padding>']
enc = en_id2char[0]
zhc = zh_id2char[0]
en_char2id['<padding>'], zh_char2id['<padding>'] = 0, 0
en_char2id[enc], zh_char2id[zhc] = eni, zhi
en_id2char[0], zh_id2char[0] = '<padding>', '<padding>'
en_id2char[eni], zh_id2char[zhi] = enc, zhc

trid_en, tsid_en =[[en_char2id[c] for c in d] for d in tr_en], [[en_char2id[c] for c in d] for d in ts_en]
trid_zh, tsid_zh =[[zh_char2id[c] for c in d] for d in tr_zh], [[zh_char2id[c] for c in d] for d in ts_zh]

trid_en, tsid_en = torch.tensor(trid_en), torch.tensor(tsid_en)
trid_zh, tsid_zh = torch.tensor(trid_zh), torch.tensor(tsid_zh)

In [ ]:
# 参数设计
emb_dim = 512
feedforward_dim = 2048
k_dim = v_dim = 64
num_layers = 6
num_heads = 8

maxen = len(trid_en[0])
# maxzh代表中文句子的长度，作为解码器的输入，需要带起始标志，而输出则不需要。为了形状统一，将句子长度-1
maxzh = len(trid_zh[0]) - 1
dtype = torch.float32


In [ ]:
print(maxen, maxzh)

52 87


In [ ]:
class PositionalEncoding(nn.Module):
  # 玛德，忘记加下划线了
  def __init__(self, model_dim=emb_dim, dropout=0.1, max_len=maxen):
    super(PositionalEncoding, self).__init__()
    self.dropout = nn.Dropout(dropout)
    
    pe = torch.zeros(max_len, model_dim)
    position = torch.arange(max_len, dtype = dtype).unsqueeze(1)
    divterm = torch.exp(torch.arange(0, model_dim, 2).float() * (-math.log(10000.0) / model_dim))
    pe[:, 0::2] = torch.sin(position * divterm)
    pe[:, 1::2] = torch.cos(position * divterm)
    # 这一步增加一维，满足x的形状
    pe = pe.unsqueeze(0).transpose(0, 1)
    # 命名为pe,forward中的self.pe源于此,更重要的是固定tensor的值，tensor的值在之后反向传播中值不会变
    self.register_buffer('pe', pe)

  def forward(self, x):
    # x的形状 (seq_len, batch_size, emb_dim)
    # 这里x的形状原先是(b, s, emb)，这样一来输入调整了一次，输出又要调回去，主要是为了照顾x可能超长
    
    x += self.pe[:x.size(0), :]
    return self.dropout(x)



In [ ]:
test = PositionalEncoding()
x = torch.zeros(maxen, emb_dim).unsqueeze(1)
test(x).shape


torch.Size([52, 1, 512])

In [ ]:
def get_attn_pad_mask(seq_q, seq_k):
  batch_size, len_q = seq_q.size()
  batch_size, len_k = seq_k.size()
  mask = seq_k.data.eq(0).unsqueeze(1)
  return mask.expand(batch_size, len_q, len_k)

In [ ]:
# tmp = torch.tensor([0, 1, 0]).unsqueeze(0)
# get_attn_pad_mask(tmp, tmp)

In [ ]:
def get_subsequence_mask(seq):
  # seq: [batch_size, len_seq]
  attn_shape = [seq.size(0), seq.size(1), seq.size(1)]
  mask = np.triu(np.ones(attn_shape), k=1)
  # 用byte是为了之后能使用masked_fill
  # bool()可以用来避免userwarning，不过之后是用来是要和其他mask相加的，可以省略
  mask = torch.from_numpy(mask).byte()
  return mask

In [ ]:
class ScaledDotProductAttention(nn.Module):
  def __init__(self):
    super(ScaledDotProductAttention, self).__init__()
  def forward(self, q, k, v, attn_mask):
    # k: [batch_size, n_heads, len_k, k_dim]
    # matmul对于多维情况，是对最后两维做矩阵乘法
    scores = torch.matmul(q, k.transpose(-1, -2)) / np.sqrt(k_dim)
    # 把scores中的1全部替换为-inf(-1e9)，scores要为byte类型
    # 下划线代表修改原tensor
    scores.masked_fill_(attn_mask, -1e9)
    attn = nn.Softmax(dim=-1)(scores)
    context = torch.matmul(attn, v)

    return context, attn

In [ ]:
# test = ScaledDotProductAttention()
# tmp = torch.randn(2, 3, 4, 5)
# sq = sk = torch.tensor([[1, 0, 0, 0],[2 , 3, 0, 0]])
# sq.to(device)
# sk.to(device)
# tmp = tmp.to(device)
# # 两种mask都测试一下
# mask = get_attn_pad_mask(sq, sk)
# mask = get_subsequence_mask(sq)
# m = mask.unsqueeze(1).repeat(1, 3, 1, 1)
# m = m.to(device)
# c, a = test(tmp, tmp, tmp, m)

# print(a.shape, c.shape)
# print(a[0, 0])

In [ ]:
class MultiheadAttention(nn.Module):
  def __init__(self, model_dim=emb_dim):
    super(MultiheadAttention, self).__init__()
    # query和key的维度要相同
    self.wq = nn.Linear(model_dim, k_dim * num_heads, bias=False)
    self.wk = nn.Linear(model_dim, k_dim * num_heads, bias=False)
    self.wv = nn.Linear(model_dim, v_dim * num_heads, bias=False)
    self.dense = nn.Linear(v_dim * num_heads, model_dim, bias=False)
    self.model_dim = model_dim
  def forward(self,input_q, input_k, input_v, mask):
    residual, batch_size = input_q, input_q.shape[0]
    # 不能直接将q修改到最终的形状而需要通过transpose，因为经过self.wq后,
    # 形状为[batch_size, len_q, num_heads * k_dim]
    q = self.wq(input_q).view(batch_size, -1, num_heads, k_dim).transpose(1, 2)
    # k,v的length相同，即下面-1的维度
    k = self.wk(input_k).view(batch_size, -1, num_heads, k_dim).transpose(1, 2)
    v = self.wv(input_v).view(batch_size, -1, num_heads, v_dim).transpose(1, 2)
    mask = mask.unsqueeze(1).repeat(1, num_heads, 1, 1)
    # 这里可以不用作为model，因为该module类并没有神经网络参数
    context, attn = ScaledDotProductAttention()(q, k, v, mask)
    # context: [batch_size, num_heads, len_q, v_dim]
    # 将context变换，以便将其输入self.dense,最终能与residual进行加和
    context = context.transpose(1, 2).reshape(batch_size, -1, v_dim * num_heads)
    output = self.dense(context)
    return nn.LayerNorm(emb_dim).cuda()(output + residual), attn

# tmp = torch.randn(2, 4, emb_dim)
# tmp = tmp.to(device)
# mask = get_attn_pad_mask(sq, sk).to(device)
# test = MultiheadAttention().to(device)
# print(tmp.device)
# iv, a = test(tmp, tmp, tmp, mask)
# print(iv.shape, a.shape)

In [ ]:
class PoswiseFeedForwardNet(nn.Module):
  def __init__(self, model_dim=emb_dim):
    super(PoswiseFeedForwardNet, self).__init__()
    self.dense = nn.Sequential(
        nn.Linear(model_dim, feedforward_dim, bias=False),
        nn.ReLU(),
        nn.Linear(feedforward_dim, model_dim, bias=False)
        )
    # self.dense = self.dense.to(device)
    self.model_dim = model_dim
  def forward(self, inputs):
    residual = inputs
    outputs = self.dense(inputs)
    return nn.LayerNorm(emb_dim).cuda()(outputs + residual)
# test = PoswiseFeedForwardNet()
# tmp1 = torch.randn(2, 4, emb_dim)
# # tmp1 = tmp1.to(device)
# test(tmp1).shape 

In [ ]:
# encoder layer，单独设一类是因为一个encoder中可以有多个encoder layer
class EncoderLayer(nn.Module):
  def __init__(self):
    super(EncoderLayer, self).__init__()
    self.attention = MultiheadAttention()
    self.feedforward = PoswiseFeedForwardNet()
  def forward(self, inputs, mask):
    outputs, attn = self.attention(inputs, inputs, inputs, mask)
    outputs = self.feedforward(outputs)
    return outputs, attn

In [ ]:
# test = EncoderLayer().to(device)
# tmp = torch.randn(2, 4, emb_dim)
# tmp = tmp.to(device)
# mask = get_subsequence_mask(torch.randn(2, 4).to(device))
# ot, a = test(tmp, mask)
# print(ot.shape, a.shape)

In [ ]:
class Encoder(nn.Module):
  def __init__(self):
    super(Encoder, self).__init__()
    # 这里是翻译任务，英语转中文，中英文的emb_dim相同
    self.en_embedding = nn.Embedding(en_vocab_size, emb_dim)
    self.pos_embedding = PositionalEncoding()
    self.layers = nn.ModuleList([EncoderLayer() for _ in range(num_layers)])

  def forward(self, inputs):
    # inputs: [batch_size, seq_len]
    en_embs = self.en_embedding(inputs)
    pos_embs = self.pos_embedding(en_embs.transpose(0, 1)).transpose(0, 1)
    mask = get_attn_pad_mask(inputs, inputs)
    outputs, attns = pos_embs, []
    for layer in self.layers:
      outputs, attn = layer(outputs, mask)
      attns.append(attn)
    return outputs, attns

# test = Encoder().to(device)
# tmp = torch.arange(6).view(2, 3)
# tmp = tmp.to(device)
# ot, a = test(tmp)
# print(ot.shape, a[0].shape)

In [ ]:
class DecoderLayer(nn.Module):
  def __init__(self):
    super(DecoderLayer, self).__init__()
    self.self_attention = MultiheadAttention()
    self.dec_enc_attention = MultiheadAttention()
    self.feedforward = PoswiseFeedForwardNet()
  
  def forward(self, dec_inputs, enc_outputs, self_mask, enc_mask):
    dec_outputs, self_attn = self.self_attention(dec_inputs, dec_inputs, dec_inputs, self_mask)
    dec_outputs, dec_enc_attn = self.dec_enc_attention(dec_outputs, enc_outputs, enc_outputs, enc_mask)
    dec_outputs = self.feedforward(dec_outputs)
    return dec_outputs, self_attn, dec_enc_attn
    

In [ ]:
class Decoder(nn.Module):
  def __init__(self, model_dim=emb_dim):
    super(Decoder, self).__init__()
    self.ch_embedding = nn.Embedding(zh_vocab_size, model_dim)
    self.pos_embedding = PositionalEncoding(max_len=maxzh)
    self.layers = nn.ModuleList([DecoderLayer() for _ in range(num_layers)])
  
  # 这里enc_inputs仅仅用于构造enc2dec
  def forward(self, dec_inputs, enc_inputs, enc_outputs):
    dec_outputs = self.ch_embedding(dec_inputs)
    dec_outputs = self.pos_embedding(dec_outputs.transpose(0, 1)).transpose(0, 1).cuda()
    # 至于遮挡到哪里，以"我很好"为例，当我提出"很"这个query时，我只能看到"我很"两个内容
    dec_pad_mask = get_attn_pad_mask(dec_inputs, dec_inputs).cuda()
    dec_seq_mask = get_subsequence_mask(dec_inputs).cuda()
    dec_mask = torch.gt((dec_pad_mask + dec_seq_mask), 0).cuda()
    # enc_mask
    enc_mask = get_attn_pad_mask(dec_inputs, enc_inputs)
    # 这里命名其实不太准确，dec_attn代表decoder的自注意力分数，enc_attn代表decoder和encoder的联合注意力分数
    dec_attns, enc_attns = [], []
    for layer in self.layers:
      dec_outputs, dec_attn, enc_attn = layer(dec_outputs, enc_outputs, dec_mask, enc_mask)
      dec_attns.append(dec_attn)
      enc_attns.append(enc_attn)
    return dec_outputs, dec_attns, enc_attns

In [ ]:
# testenc = Encoder().cuda()
# testdec = Decoder().cuda()
# # enc = torch.arange(6).view(2, 3)
# # dec = torch.arange(8).view(2, 4)

# dec = trid_zh[:2, :]
# enc = trid_en[:2, :]
# enc_ot, ea = testenc(enc)
# # enc_ot = enc_ot.to(device)
# dec_ot, da, eda = testdec(dec, enc, enc_ot)

In [ ]:
class Transformer(nn.Module):
  def __init__(self, model_dim = emb_dim):
    super(Transformer,self).__init__()
    self.encoder = Encoder().cuda()
    self.decoder = Decoder().cuda()
    self.projection = nn.Linear(model_dim, zh_vocab_size, bias=False).cuda()
  def forward(self, enc_inputs, dec_inputs):
    enc_outputs, enc_self_attns = self.encoder(enc_inputs)
    # enc_outputs = enc_outputs.to(device)
    dec_outputs, dec_self_attns, dec_enc_attns = self.decoder(dec_inputs, enc_inputs, enc_outputs)
    dec_logits = self.projection(dec_outputs)
    
    return dec_logits.view(-1, dec_logits.shape[-1]), enc_self_attns, dec_self_attns, dec_enc_attns

In [ ]:
# test = Transformer().to(device)
# enc = torch.arange(6).view(2, 3)
# dec = torch.arange(8).view(2, 4)
# enc = enc.to(device)
# dec = dec.to(device)

# dl, ea, da, dea = test(enc, dec)

In [ ]:
# 将预测结果以好观察的形式呈现
def sentence(lis):
  j = len(lis) - 1
  while lis[j] == '<padding>' or lis[j] == '<EOS>':
    j -= 1
    if j == -1:
      return ''
  return ''.join(lis[:j+1])
def translate(model, tri, tsi):
  tren,tsen = trid_en[tri].unsqueeze(0), tsid_en[tsi].unsqueeze(0)
  trzh, tszh = trid_zh[tri].unsqueeze(0), tsid_zh[tsi].unsqueeze(0)
  tren, tsen, trzh, tszh = tren.cuda(), tsen.cuda(), trzh.cuda(), tszh.cuda()
  tro, ea, da, eda = model(tren, trzh[:, :-1])
  tso, ea, da, eda = model(tsen, tszh[:, :-1])
  tro1, tso1 = tro.squeeze(1).argmax(1), tso.squeeze(1).argmax(1)
  tro2, tso2 = [zh_id2char[i] for i in tro1], [zh_id2char[i] for i in tso1]
  print('train sentence:', sentence(tro2))
  print('test sentence:', sentence(tso2))



In [ ]:
torch.cuda.empty_cache()

In [ ]:
# 开始训练
num_epochs = 50
batch_size = 128
loss = nn.CrossEntropyLoss(ignore_index=0)

# 注，这里emb_dim编码器解码器共用

model = Transformer().cuda()
# optimizer = torch.optim.Adam(model.parameters(), lr=3e-3)
optimizer = torch.optim.SGD(model.parameters(), lr=3e-3, momentum=0.99)
tri, tsi = 10, 10
print('correct train', sentence(tr_zh[tri]))
print('correct test', sentence(ts_zh[tsi]))
for epoch in range(num_epochs):
  start, loss_sum, n = time.time(), 0.0, 0
  dataset = torch.utils.data.TensorDataset(trid_en, trid_zh)
  train_iter = torch.utils.data.DataLoader(dataset, batch_size, shuffle=True)
  for tren, trzh in train_iter:
    tren, trzh = tren.cuda(), trzh.cuda()
    enc_inputs = tren
    dec_inputs = trzh[:, :-1]
    dec_outputs, enc_self_attns, dec_self_attns, dec_enc_attns = model(enc_inputs, dec_inputs)
    # dec_inputs的ground truth相对于标签是要左移一位的，
    # 相当于dec_inputs "<SOS>我很好" 对应的表签y "我很好<EOS>"
    y = trzh[:, 1:]
    # 对比sq2sq，模型输出的结果是逐个时间步产生的，因此需要transpose，
    # 而transformer只是在self attention是逐个时间步计算，最终输出是全序列一次性输出，所以不用transpose
    yt = y.contiguous().view(-1)
    l = loss(dec_outputs, yt.long())
    # 训练三部曲
    optimizer.zero_grad()
    l.backward()
    optimizer.step()

    loss_sum += l.item() * trzh.shape[0]
    n += trzh.shape[0]
  try:
    perplexity = math.exp(loss_sum / n)
  except OverflowError:
    perplexity = float('inf')
  print("epoch:{} perplexity:{} time:{}".format(epoch, perplexity, time.time() - start))
  translate(model, tri, tsi)



correct train <SOS>播用跑道号码更是幼稚得可怜用5左5右这种令人混乱的数字为什么不采用5678等等
correct test <SOS>在西伯伦市中心的犹太人定居点外巴勒斯坦示威者与警察发生了冲突
epoch:0 perplexity:7792.580768721553 time:61.98250985145569
train sentence: 是是是是<EOS>是<EOS>是是是是<EOS>是<EOS><EOS>一的<EOS>的<EOS>是<EOS><EOS>是
test sentence: 是是是是是是是是一<EOS><EOS><EOS>是是
epoch:1 perplexity:3580.3433243222144 time:61.498366594314575
train sentence: 是是是是的是的的的的的的的的的的的的的了的的的的的的的的的
test sentence: 是是是是是的的的的的的的的的的
epoch:2 perplexity:2656.17660833955 time:61.20496463775635
train sentence: 的的的的的的的的的的的的的的的的的的的的的的的的的的的的的
test sentence: 的的的的的的的的的的的的的的的
epoch:3 perplexity:2278.9264785740397 time:61.18912220001221
train sentence: 他的的的的的的的的的的的的的的的的的的的的的的的的的的的的
test sentence: 他的的的的的的的的的的的的的的
epoch:4 perplexity:2068.2278818874634 time:61.16396689414978
train sentence: 在在在在的的的的的的的的的的的的的的的一的的的的的的的的的
test sentence: 在在在的一的的的的的的的的的的
epoch:5 perplexity:1926.0342222963113 time:61.200403928756714
train sentence: 在在在在的的的的的的的的的的的的的的的一的的的的的的的的的
test sentence: 在这在的一的的的的的的的的的的
epoch:6 perplexity:1822

In [ ]:
sentences = [
        # enc_input           dec_input         dec_output
        ['ich mochte ein bier P', 'S i want a beer .', 'i want a beer . E'],
        ['ich mochte ein cola P', 'S i want a coke .', 'i want a coke . E']
]

# Padding Should be Zero
src_vocab = {'P' : 0, 'ich' : 1, 'mochte' : 2, 'ein' : 3, 'bier' : 4, 'cola' : 5}
src_vocab_size = len(src_vocab)

tgt_vocab = {'P' : 0, 'i' : 1, 'want' : 2, 'a' : 3, 'beer' : 4, 'coke' : 5, 'S' : 6, 'E' : 7, '.' : 8}
idx2word = {i: w for i, w in enumerate(tgt_vocab)}
tgt_vocab_size = len(tgt_vocab)

src_len = 5 # enc_input max sequence length
tgt_len = 6 # dec_input(=dec_output) max sequence length

def make_data(sentences):
    enc_inputs, dec_inputs, dec_outputs = [], [], []
    for i in range(len(sentences)):
      enc_input = [[src_vocab[n] for n in sentences[i][0].split()]] # [[1, 2, 3, 4, 0], [1, 2, 3, 5, 0]]
      dec_input = [[tgt_vocab[n] for n in sentences[i][1].split()]] # [[6, 1, 2, 3, 4, 8], [6, 1, 2, 3, 5, 8]]
      dec_output = [[tgt_vocab[n] for n in sentences[i][2].split()]] # [[1, 2, 3, 4, 8, 7], [1, 2, 3, 5, 8, 7]]

      enc_inputs.extend(enc_input)
      dec_inputs.extend(dec_input)
      dec_outputs.extend(dec_output)

    return torch.LongTensor(enc_inputs), torch.LongTensor(dec_inputs), torch.LongTensor(dec_outputs)

enc_inputs, dec_inputs, dec_outputs = make_data(sentences)

class MyDataSet(Data.Dataset):
  def __init__(self, enc_inputs, dec_inputs, dec_outputs):
    super(MyDataSet, self).__init__()
    self.enc_inputs = enc_inputs
    self.dec_inputs = dec_inputs
    self.dec_outputs = dec_outputs
  
  def __len__(self):
    return self.enc_inputs.shape[0]
  
  def __getitem__(self, idx):
    return self.enc_inputs[idx], self.dec_inputs[idx], self.dec_outputs[idx]

loader = Data.DataLoader(MyDataSet(enc_inputs, dec_inputs, dec_outputs), 2, True)

In [ ]:
model = Transformer().cuda()
criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.SGD(model.parameters(), lr=1e-3, momentum=0.99)
for epoch in range(30):
    for enc_inputs, dec_inputs, dec_outputs in loader:
      '''
      enc_inputs: [batch_size, src_len]
      dec_inputs: [batch_size, tgt_len]
      dec_outputs: [batch_size, tgt_len]
      '''
      enc_inputs, dec_inputs, dec_outputs = enc_inputs.cuda(), dec_inputs.cuda(), dec_outputs.cuda()
      # outputs: [batch_size * tgt_len, tgt_vocab_size]
      outputs, enc_self_attns, dec_self_attns, dec_enc_attns = model(enc_inputs, dec_inputs)
      loss = criterion(outputs, dec_outputs.view(-1))
      print('Epoch:', '%04d' % (epoch + 1), 'loss =', '{:.6f}'.format(loss))

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

Epoch: 0001 loss = 9.980536
Epoch: 0002 loss = 9.589383
Epoch: 0003 loss = 9.100897
Epoch: 0004 loss = 8.223795
Epoch: 0005 loss = 7.306498
Epoch: 0006 loss = 6.316115
Epoch: 0007 loss = 5.424812
Epoch: 0008 loss = 4.721016
Epoch: 0009 loss = 4.059168
Epoch: 0010 loss = 3.496286
Epoch: 0011 loss = 2.962638
Epoch: 0012 loss = 2.541767
Epoch: 0013 loss = 2.162373
Epoch: 0014 loss = 1.815699
Epoch: 0015 loss = 1.551307
Epoch: 0016 loss = 1.381725
Epoch: 0017 loss = 1.218789
Epoch: 0018 loss = 1.108609
Epoch: 0019 loss = 0.966831
Epoch: 0020 loss = 0.831889
Epoch: 0021 loss = 0.679625
Epoch: 0022 loss = 0.523888
Epoch: 0023 loss = 0.438625
Epoch: 0024 loss = 0.397825
Epoch: 0025 loss = 0.374220
Epoch: 0026 loss = 0.368728
Epoch: 0027 loss = 0.351252
Epoch: 0028 loss = 0.285497
Epoch: 0029 loss = 0.208163
Epoch: 0030 loss = 0.143371
